In [1]:
import numpy as np
from flask import Flask, request, render_template
import pickle
import warnings
warnings.simplefilter('ignore')

In [2]:
league = np.loadtxt('league.txt', dtype=str, delimiter=',')
nation = np.loadtxt('nation.txt', dtype=str, delimiter=',')
custom_order = ['GK', 'LWB', 'LB', 'CB', 'RB', 'RWB', 'CDM', 'CM', 'LM', 'RM', 'CAM', 'LW', 'RW', 'CF', 'ST']

In [ ]:
#Create an app object using the Flask class. 
app = Flask(__name__)

#Load the trained model. (Pickle file)
model = pickle.load(open('model.pkl', 'rb'))

#Define the route to be home. 
#The decorator below links the relative route of the URL to the function it is decorating.
#Here, home function is with '/', our root directory. 
#Running the app sends us to index.html.
#Note that render_template means it looks for the file in the templates folder. 

#use the route() decorator to tell Flask what URL should trigger our function.
@app.route('/')
def home():
    return render_template('index.html', league = league, nation = nation, position = custom_order)

#You can use the methods argument of the route() decorator to handle different HTTP methods.
#GET: A GET message is send, and the server returns data
#POST: Used to send HTML form data to the server.
#Add Post method to the decorator to allow for form submission. 
#Redirect to /predict page with the output
@app.route('/predict',methods=['POST'])
def predict():
    all_features = [str(x) for x in request.form.values()]
    number_features = all_features[:12]
    features = [float(x) for x in number_features]
    for i in league:
        if i == all_features[12]:
            features.append(1)
        else:
            features.append(0)
    for i in nation:
        if i == all_features[13]:
            features.append(1)
        else:
            features.append(0)
    for i in custom_order:
        if np.isin(i, all_features[14:]):
            features.append(1)
        else:
            features.append(0)
    features = [features]
    prediction = model.predict(features)  # features Must be in the form [[a, b]]
    output = round(prediction[0], 2)
    return render_template('index.html', prediction_text='Price is {}'.format(output))


#When the Python interpreter reads a source file, it first defines a few special variables. 
#For now, we care about the __name__ variable.
#If we execute our code in the main program, like in our case here, it assigns
# __main__ as the name (__name__). 
#So if we want to run our code right here, we can check if __name__ == __main__
#if so, execute it here. 
#If we import this file (module) to another file then __name__ == app (which is the name of this python file).

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/Dec/2023 13:50:12] "GET / HTTP/1.1" 200 -
